# Test interrogating local Rust docs with an LLM

See: [Talk to your Text files in Vector Databases with GPT-4 and ChromaDB](https://medium.com/@rubentak/unleashing-the-power-of-intelligent-chatbots-with-gpt-4-and-vector-databases-a-step-by-step-8027e2ce9e78)

Import libraries

In [2]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

Initialize the basic parameters needed for Chroma, considering that the data already exists on disk

In [3]:
persist_directory = 'db'

# OpenAI embeddings
embedding = OpenAIEmbeddings()

Now we can load the persisted database from disk

In [4]:
vectordb = Chroma(persist_directory=persist_directory,
                   embedding_function=embedding)

In order to get the data out of the database again, we need to create a retriever. This retriever will return all the documents (or chucks) related to the question asked:

In [5]:
# Create retriever
retriever = vectordb.as_retriever()

docs = retriever.get_relevant_documents("How can I read a file in Rust?")

docs

[Document(page_content='Reading a File\n\nNow we’ll add functionality to read the file specified in the file_path\nargument. First, we need a sample file to test it with: we’ll use a file with a\nsmall amount of text over multiple lines with some repeated words. Listing 12-3\nhas an Emily Dickinson poem that will work well! Create a file called\npoem.txt at the root level of your project, and enter the poem “I’m Nobody!\nWho are you?”\n\nFilename: poem.txt\n\ntext\n{{#include ../listings/ch12-an-io-project/listing-12-03/poem.txt}}\n\nListing 12-3: A poem by Emily Dickinson makes a good test\ncase\n\nWith the text in place, edit src/main.rs and add code to read the file, as\nshown in Listing 12-4.\n\nFilename: src/main.rs\n\nrust,should_panic,noplayground\n{{#rustdoc_include ../listings/ch12-an-io-project/listing-12-04/src/main.rs:here}}\n\nListing 12-4: Reading the contents of the file specified\nby the second argument\n\nFirst, we bring in a relevant part of the standard library with 

## Create a Q&A chain
The chain will be created. Consider it to be akin to the prompt in ChatGPT.

In [6]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

To see how our chatbot will come to the answers that it returns, we can create the following function:

In [7]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

Let's repeat the previous query:

In [10]:
query = "How can I read a file in Rust?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 You can use the fs::read_to_string function from the standard library to read the contents of a file into a String.


Sources:
D:\src\pareidolia\book\ch12-02-reading-a-file.md
D:\src\pareidolia\book\ch09-02-recoverable-errors-with-result.md
D:\src\pareidolia\book\ch12-02-reading-a-file.md
D:\src\pareidolia\book\ch12-02-reading-a-file.md


In [11]:
query = "Give an example of how to read a file in Rust?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 Chapter 12 of the Rust book explains how to read a file in Rust. The example code from the book is shown below: 
{{#rustdoc_include ../listings/ch12-an-io-project/listing-12-04/src/main.rs}}


Sources:
D:\src\pareidolia\book\ch00-00-introduction.md
D:\src\pareidolia\book\ch14-04-installing-binaries.md
D:\src\pareidolia\book\ch00-00-introduction.md
D:\src\pareidolia\book\ch12-02-reading-a-file.md


In [12]:
query = "What is a dymanic trait. I am not familiar with this. Is it new? "
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 Dynamic dispatch is when the compiler can’t tell at compile time which method you’re calling. Instead, at runtime, Rust uses the pointers inside the trait object to know which method to call. This lookup incurs a runtime cost that doesn’t occur with static dispatch. Dynamic dispatch is not new; it has been around for a while.


Sources:
D:\src\pareidolia\book\ch17-02-trait-objects.md
D:\src\pareidolia\book\ch17-02-trait-objects.md
D:\src\pareidolia\book\ch19-03-advanced-traits.md
D:\src\pareidolia\book\ch10-02-traits.md


In [13]:
query = "Is Rust difficult? How does it compare with Java? Or with Python?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 
Rust is not difficult, however it does have a different approach to programming than Java or Python. Rust has a strong focus on memory safety and provides tools to help developers write reliable code. Java and Python are both high-level programming languages, but Rust allows developers to have more control over memory usage and performance.


Sources:
D:\src\pareidolia\book\foreword.md
D:\src\pareidolia\book\foreword.md
D:\src\pareidolia\book\ch00-00-introduction.md
D:\src\pareidolia\book\SUMMARY.md


In [7]:
query = "How can I speed up compiling?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 You can use the command cargo check to quickly check your code to make sure it compiles, but does not produce an executable. This is often much faster than cargo build, and can be used to check your code when writing it to make sure it compiles.


Sources:
D:\src\pareidolia\book\ch01-03-hello-cargo.md
D:\src\pareidolia\book\ch01-02-hello-world.md
D:\src\pareidolia\book\ch01-02-hello-world.md
D:\src\pareidolia\book\ch01-03-hello-cargo.md


[Now we have to implement conversational memory...](https://python.langchain.com/docs/modules/memory/how_to/conversational_customization)

In [10]:
query = "Can you show how to start a parralel task in Rust? Give an example."
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 Yes! Rust offers several tools to help you work with concurrency and parallelism. An example of starting a parallel task in Rust is to use the Rayon crate. You can use the Rayon crate to create Rayon "thread pools" and spawn tasks on those thread pools. For example, this code will spawn a task on a Rayon thread pool: 
let thread_pool = Rayon::new().unwrap();
thread_pool.spawn(|| {
    // Do something in the thread
});


Sources:
D:\src\pareidolia\book\ch16-00-concurrency.md
D:\src\pareidolia\book\foreword.md
D:\src\pareidolia\book\ch00-00-introduction.md
D:\src\pareidolia\book\ch16-00-concurrency.md
